In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
from sklearn.decomposition import TruncatedSVD
%matplotlib inline
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"

In [3]:
# =====================
# Settings
# =====================
SAVE_PATH = "../output/fe/fe005.feather"

In [4]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [5]:
train.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),1251960,61971,105808,718161,71513,26678,329800
1,16,Hachimitsu to Clover,8.06,"Comedy, Drama, Josei, Romance, Slice of Life",Honey and Clover,ハチミツとクローバー,TV,24,"Apr 15, 2005 to Sep 27, 2005",Spring 2005,"Genco, Fuji TV, Shueisha","VIZ Media, Discotek Media",J.C.Staff,Manga,23 min. per ep.,PG-13 - Teens 13 or older,214499,4101,11909,81145,11901,11026,98518
2,22,Tennis no Ouji-sama,7.90,"Action, Comedy, Sports, School, Shounen",The Prince of Tennis,テニスの王子様,TV,178,"Oct 10, 2001 to Mar 23, 2005",Fall 2001,"Production I.G, Nihon Ad Systems",VIZ Media,Trans Arts,Manga,22 min. per ep.,PG-13 - Teens 13 or older,141832,3124,11235,76881,12905,12516,28295
3,23,Ring ni Kakero 1,6.38,"Action, Shounen, Sports",Unknown,リングにかけろ１,TV,12,"Oct 6, 2004 to Dec 15, 2004",Fall 2004,Unknown,Unknown,Toei Animation,Manga,25 min. per ep.,PG - Children,3648,16,170,1333,145,296,1704
4,47,Akira,8.17,"Action, Military, Sci-Fi, Adventure, Horror, S...",AKIRA,AKIRA（アキラ）,Movie,1,"Jul 16, 1988",Unknown,"Mainichi Broadcasting System, Kodansha, Toho","Funimation, Bandai Entertainment, Geneon Enter...",Tokyo Movie Shinsha,Manga,2 hr. 4 min.,R+ - Mild Nudity,566538,9555,8258,434086,3491,2174,118529


In [6]:
cols = ["Producers"]
train = pd.concat([train[cols],test[cols]]).reset_index(drop=True)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tv = TfidfVectorizer()
X = tv.fit_transform(train["Producers"])
X = pd.DataFrame(X.todense())

In [14]:
n_components = 20
svd = TruncatedSVD(n_components=n_components)
X = svd.fit_transform(X)
df = pd.DataFrame(X, columns=[f"Producers_{i}" for i in range(n_components)])

In [15]:
df

,Producers_0,Producers_1,Producers_2,Producers_3,Producers_4,Producers_5,Producers_6,Producers_7,Producers_8,Producers_9,Producers_10,Producers_11,Producers_12,Producers_13,Producers_14,Producers_15,Producers_16,Producers_17,Producers_18,Producers_19
0,-5.037242e-12,2.717036e-01,8.918236e-01,-7.698482e-03,-1.374394e-01,-1.622213e-01,2.003912e-01,-2.377531e-02,3.683079e-02,2.242465e-02,-5.652146e-02,6.426833e-02,-7.482216e-02,-6.195800e-02,-3.381081e-02,4.036132e-02,1.080276e-02,8.837537e-03,-7.780750e-03,-7.498707e-03
1,4.577958e-11,4.464290e-01,-1.470410e-01,-1.758854e-03,-2.020541e-02,-5.913508e-02,2.417928e-02,-3.282169e-02,8.545491e-02,1.427138e-01,5.410070e-02,-5.196852e-02,-2.284845e-01,3.581569e-01,-9.964337e-02,1.633766e-01,1.446497e-02,2.837117e-01,1.307480e-01,-5.433430e-02
2,-3.250938e-12,8.728684e-02,-1.275813e-03,3.356477e-05,2.507103e-02,2.615572e-02,-5.327801e-02,3.413348e-02,4.919830e-03,4.966376e-02,3.538707e-02,3.337504e-02,4.114885e-02,-6.206091e-02,-1.044276e-01,1.503115e-02,1.932355e-01,-2.450164e-01,1.790479e-01,6.933711e-01
3,1.000000e+00,-1.222497e-15,-3.255296e-16,1.074234e-14,-5.312585e-15,3.326440e-15,-1.080081e-15,-6.303467e-18,-1.528831e-15,5.136040e-15,3.560988e-15,-6.877451e-15,-1.337772e-14,-1.018844e-14,1.034611e-15,7.292827e-15,3.307985e-15,3.586175e-15,-5.484133e-15,1.404309e-15
4,1.142578e-10,8.837699e-02,4.960471e-02,-6.567672e-05,1.135336e-01,1.749358e-01,-7.163700e-02,5.427347e-02,1.276897e-01,5.412697e-02,-9.157116e-02,2.524781e-01,-9.900166e-02,6.693756e-02,2.469275e-01,-2.066238e-01,1.661872e-03,-1.468159e-01,-1.997041e-01,1.192651e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12416,1.000000e+00,-5.279381e-16,2.203248e-16,-1.785497e-16,-8.583187e-16,4.344127e-16,-6.220768e-17,-4.501268e-16,-1.498935e-15,1.208699e-15,-3.762788e-16,-1.223902e-15,-2.627773e-15,-2.128361e-15,-1.655017e-15,1.061154e-15,1.843955e-15,1.822843e-15,-3.796077e-15,8.056328e-16
12417,1.000000e+00,-5.279381e-16,2.203247e-16,-1.785497e-16,-8.583186e-16,4.344130e-16,-6.220824e-17,-4.501243e-16,-1.498935e-15,1.208701e-15,-3.762771e-16,-1.223897e-15,-2.627777e-15,-2.128346e-15,-1.655025e-15,1.061151e-15,1.843941e-15,1.822849e-15,-3.796080e-15,8.056570e-16
12418,1.000000e+00,-5.279281e-16,2.203455e-16,-1.787121e-16,-8.582470e-16,4.343764e-16,-6.221084e-17,-4.500725e-16,-1.498947e-15,1.208706e-15,-3.762747e-16,-1.223835e-15,-2.627609e-15,-2.128332e-15,-1.654992e-15,1.060976e-15,1.843367e-15,1.822382e-15,-3.796105e-15,8.060027e-16
12419,1.000000e+00,-5.279203e-16,2.203607e-16,-1.788895e-16,-8.581833e-16,4.342937e-16,-6.224079e-17,-4.500665e-16,-1.498950e-15,1.208587e-15,-3.761792e-16,-1.223625e-15,-2.627355e-15,-2.128372e-15,-1.654839e-15,1.061046e-15,1.843226e-15,1.822484e-15,-3.796068e-15,8.060263e-16


In [16]:
df = reduce_mem_usage(df)
df.to_feather(SAVE_PATH)

Memory usage of dataframe is 1.90 MB
column =  20
0
Memory usage after optimization is: 0.95 MB
Decreased by 50.0%
